In [86]:
import pandas as pd
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
#from sklearn.feature_extraction.text import CountVectorizer  #DT does not take strings as input for the model fit step....
from IPython.display import Image  
#import pydotplus as pydot
from sklearn import tree
from os import system
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [3]:
ourdata = pd.read_csv("E:\Full Stack Data Scientist Bootcamp\Supervised ML resources\Bank_Personal_Loan_Modelling.csv")
ourdata.head(10) #several missing values!

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1
5,6,37,13,29,92121,4,0.4,2,155,0,0,0,1,0
6,7,53,27,72,91711,2,1.5,2,0,0,0,0,1,0
7,8,50,24,22,93943,1,0.3,3,0,0,0,0,0,1
8,9,35,10,81,90089,3,0.6,2,104,0,0,0,1,0
9,10,34,9,180,93023,1,8.9,3,0,1,0,0,0,0


In [4]:
ourdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  5000 non-null   int64  
 1   Age                 5000 non-null   int64  
 2   Experience          5000 non-null   int64  
 3   Income              5000 non-null   int64  
 4   ZIP Code            5000 non-null   int64  
 5   Family              5000 non-null   int64  
 6   CCAvg               5000 non-null   float64
 7   Education           5000 non-null   int64  
 8   Mortgage            5000 non-null   int64  
 9   Personal Loan       5000 non-null   int64  
 10  Securities Account  5000 non-null   int64  
 11  CD Account          5000 non-null   int64  
 12  Online              5000 non-null   int64  
 13  CreditCard          5000 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 547.0 KB


In [5]:
ourdata.isnull()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4996,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4997,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4998,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [6]:
ourdata.isnull().sum()

ID                    0
Age                   0
Experience            0
Income                0
ZIP Code              0
Family                0
CCAvg                 0
Education             0
Mortgage              0
Personal Loan         0
Securities Account    0
CD Account            0
Online                0
CreditCard            0
dtype: int64

In [7]:
ourdata.describe()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000
mean,2500.500000,45.338400,20.104600,73.774200,93152.503000,2.396400,1.937938,1.881000,56.498800,0.096000,0.104400,0.06040,0.596800,0.294000
std,1443.520003,11.463166,11.467954,46.033729,2121.852197,1.147663,1.747659,0.839869,101.713802,0.294621,0.305809,0.23825,0.490589,0.455637
min,1.000000,23.000000,-3.000000,8.000000,9307.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,1250.750000,35.000000,10.000000,39.000000,91911.000000,1.000000,0.700000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
50%,2500.500000,45.000000,20.000000,64.000000,93437.000000,2.000000,1.500000,2.000000,0.000000,0.000000,0.000000,0.00000,1.000000,0.000000
75%,3750.250000,55.000000,30.000000,98.000000,94608.000000,3.000000,2.500000,3.000000,101.000000,0.000000,0.000000,0.00000,1.000000,1.000000
max,5000.000000,67.000000,43.000000,224.000000,96651.000000,4.000000,10.000000,3.000000,635.000000,1.000000,1.000000,1.00000,1.000000,1.000000


In [8]:
ourdata.corr()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
ID,1.000000,-0.008473,-0.008326,-0.017695,0.013432,-0.016797,-0.024675,0.021463,-0.013920,-0.024801,-0.016972,-0.006909,-0.002528,0.017028
Age,-0.008473,1.000000,0.994215,-0.055269,-0.029216,-0.046418,-0.052012,0.041334,-0.012539,-0.007726,-0.000436,0.008043,0.013702,0.007681
Experience,-0.008326,0.994215,1.000000,-0.046574,-0.028626,-0.052563,-0.050077,0.013152,-0.010582,-0.007413,-0.001232,0.010353,0.013898,0.008967
Income,-0.017695,-0.055269,-0.046574,1.000000,-0.016410,-0.157501,0.645984,-0.187524,0.206806,0.502462,-0.002616,0.169738,0.014206,-0.002385
ZIP Code,0.013432,-0.029216,-0.028626,-0.016410,1.000000,0.011778,-0.004061,-0.017377,0.007383,0.000107,0.004704,0.019972,0.016990,0.007691
Family,-0.016797,-0.046418,-0.052563,-0.157501,0.011778,1.000000,-0.109275,0.064929,-0.020445,0.061367,0.019994,0.014110,0.010354,0.011588
CCAvg,-0.024675,-0.052012,-0.050077,0.645984,-0.004061,-0.109275,1.000000,-0.136124,0.109905,0.366889,0.015086,0.136534,-0.003611,-0.006689
Education,0.021463,0.041334,0.013152,-0.187524,-0.017377,0.064929,-0.136124,1.000000,-0.033327,0.136722,-0.010812,0.013934,-0.015004,-0.011014
Mortgage,-0.013920,-0.012539,-0.010582,0.206806,0.007383,-0.020445,0.109905,-0.033327,1.000000,0.142095,-0.005411,0.089311,-0.005995,-0.007231
Personal Loan,-0.024801,-0.007726,-0.007413,0.502462,0.000107,0.061367,0.366889,0.136722,0.142095,1.000000,0.021954,0.316355,0.006278,0.002802


In [9]:
ourdata.dtypes

ID                      int64
Age                     int64
Experience              int64
Income                  int64
ZIP Code                int64
Family                  int64
CCAvg                 float64
Education               int64
Mortgage                int64
Personal Loan           int64
Securities Account      int64
CD Account              int64
Online                  int64
CreditCard              int64
dtype: object

In [10]:
ourdata.columns

Index(['ID', 'Age', 'Experience', 'Income', 'ZIP Code', 'Family', 'CCAvg',
       'Education', 'Mortgage', 'Personal Loan', 'Securities Account',
       'CD Account', 'Online', 'CreditCard'],
      dtype='object')

In [11]:

print (ourdata.value_counts())

ID    Age  Experience  Income  ZIP Code  Family  CCAvg  Education  Mortgage  Personal Loan  Securities Account  CD Account  Online  CreditCard
1     25   1           49      91107     4       1.6    1          0         0              1                   0           0       0             1
3331  34   9           32      95054     4       1.1    3          0         0              0                   0           1       0             1
3338  59   29          61      92008     3       2.0    3          0         0              0                   0           1       0             1
3337  60   34          11      94305     4       0.7    3          0         0              0                   0           1       0             1
3336  35   10          118     92069     2       7.8    1          358       0              0                   0           0       0             1
                                                                                                                     

In [12]:
ourdata['Personal Loan'].value_counts()

0    4520
1     480
Name: Personal Loan, dtype: int64

In [13]:
# Dividing the data into the dependent and independent data
y=ourdata['Personal Loan']
y

0       0
1       0
2       0
3       0
4       0
       ..
4995    0
4996    0
4997    0
4998    0
4999    0
Name: Personal Loan, Length: 5000, dtype: int64

In [14]:
x=ourdata.drop('Personal Loan', axis=1)

In [15]:
# Standries the data 
from scipy.stats import zscore
Xscaled=x.apply(zscore)
Xscaled

x=pd.DataFrame(Xscaled)
x

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Securities Account,CD Account,Online,CreditCard
0,-1.731704,-1.774417,-1.666078,-0.538229,-0.964114,1.397414,-0.193385,-1.049078,-0.555524,2.928915,-0.25354,-1.216618,-0.645314
1,-1.731012,-0.029524,-0.096330,-0.864109,-1.443932,0.525991,-0.250611,-1.049078,-0.555524,2.928915,-0.25354,-1.216618,-0.645314
2,-1.730319,-0.552992,-0.445163,-1.363793,0.738814,-1.216855,-0.536736,-1.049078,-0.555524,-0.341423,-0.25354,-1.216618,-0.645314
3,-1.729626,-0.901970,-0.968413,0.569765,0.452243,-1.216855,0.436091,0.141703,-0.555524,-0.341423,-0.25354,-1.216618,-0.645314
4,-1.728933,-0.901970,-1.055621,-0.625130,-0.859007,1.397414,-0.536736,0.141703,-0.555524,-0.341423,-0.25354,-1.216618,1.549632
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,1.728933,-1.425438,-1.491662,-0.733757,-0.214694,-1.216855,-0.021710,1.332484,-0.555524,-0.341423,-0.25354,0.821951,-0.645314
4996,1.729626,-1.338194,-1.404454,-1.276892,-0.525774,1.397414,-0.880087,-1.049078,0.280238,-0.341423,-0.25354,0.821951,-0.645314
4997,1.730319,1.540880,1.647835,-1.081363,-0.061039,-0.345432,-0.937312,1.332484,-0.555524,-0.341423,-0.25354,-1.216618,-0.645314
4998,1.731012,1.715370,1.735043,-0.538229,-1.469855,0.525991,-0.822862,0.141703,-0.555524,-0.341423,-0.25354,0.821951,-0.645314


In [16]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.30,random_state=0)

#### Applying the KNN Classifier

In [28]:
from sklearn.neighbors import KNeighborsClassifier
model_1=KNeighborsClassifier(n_neighbors=10,weights='distance')

model_1.fit(x_train,y_train)

print(model_1.score(x_train,y_train))
print(model_1.score(x_test,y_test))

y_pred_m1=model_1.predict(x_test)

1.0
0.958


#### Appling the Navie Bayes Model

In [34]:
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB

# fit the model on the train data set
model_2=GaussianNB()
model_2.fit(x_train,y_train)

print(model_2.score(x_train,y_train))
print(model_2.score(x_test,y_test))

0.8825714285714286
0.8853333333333333


#### Appling the SVM Classifier

In [35]:
from sklearn import svm
model_3=svm.SVC(C=3)

model_3.fit(x_train,y_train)

print(model_3.score(x_train,y_train))
print(model_3.score(x_test,y_test))

0.9891428571428571
0.9793333333333333


#### Applying the Decision tree model

In [45]:
model_4= DecisionTreeClassifier(criterion = 'gini', max_depth=3 ,random_state=1)
model_4.fit(x_train, y_train)

print(model_4.score(x_train,y_train))
print(model_4.score(x_test,y_test))

0.9797142857142858
0.9813333333333333


#### Applying the Bagging Alogorithm

In [47]:
from sklearn.ensemble import BaggingClassifier

# model_5 = BaggingClassifier(base_estimator=model_4, n_estimators=50,random_state=1)
model_5 = BaggingClassifier(n_estimators=50,random_state=1)

model_5.fit(x_train, y_train)

print(model_5.score(x_train,y_train))
print(model_5.score(x_test,y_test))

0.9997142857142857
0.9853333333333333


#### Appling the AdaBoost Algoritum

In [49]:
from sklearn.ensemble import AdaBoostClassifier

# model_6 = AdaBoostClassifier(base_estimator=model_4, n_estimators=10, random_state=1)
model_6 = AdaBoostClassifier( n_estimators=50,random_state=1)
model_6.fit(x_train, y_train)

print(model_5.score(x_train,y_train))
print(model_5.score(x_test,y_test))

0.9997142857142857
0.9853333333333333


#### Appling the Gradient Boosting

In [55]:
from sklearn.ensemble import GradientBoostingClassifier


model_7 = GradientBoostingClassifier(n_estimators = 50,random_state=1)

model_7.fit(x_train, y_train)

print(model_7.score(x_train,y_train))
print(model_7.score(x_test,y_test))

0.99
0.986


#### Appling the Xtream Gradient Boosting

In [65]:
!pip install xgboost

In [67]:
from xgboost import XGBClassifier

model_8 = XGBClassifier() 
model_8.fit(x_train, y_train)

print(model_8.score(x_train,y_train))
print(model_8.score(x_test,y_test))

1.0
0.9873333333333333


#### Appling the Random Forest Algorithm

In [68]:
from sklearn.ensemble import RandomForestClassifier

model_9 = RandomForestClassifier(n_estimators = 50, random_state=1,max_features=12)
model_9.fit(x_train, y_train)

print(model_9.score(x_train,y_train))
print(model_9.score(x_test,y_test))

0.9997142857142857
0.9853333333333333


#### Appling the Stacking 

In [90]:
from sklearn.ensemble import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier

dtc =  DecisionTreeClassifier()
rfc = RandomForestClassifier()
knn =  KNeighborsClassifier()
xgb = XGBClassifier()
clf = [('dtc',dtc),('rfc',rfc),('knn',knn),('xgb',xgb)] #list of (str, estimator)

lr = LogisticRegression()

stack_model = StackingClassifier( estimators = clf,final_estimator = lr)

score = cross_val_score(stack_model,x,y,cv = 5,scoring = 'accuracy')

# print(score.score(x_train,y_train))
# print(score.score(x_test,y_test))
print("The final accuracy score is: ",score.mean())

The final accuracy score is:  0.9864


In [77]:
!pip install xgboost

In [88]:
from sklearn.ensemble import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier

dtc =  DecisionTreeClassifier()
rfc = RandomForestClassifier()
knn =  KNeighborsClassifier()
xgb = XGBClassifier()

clf = [('dtc',dtc),('rfc',rfc),('knn',knn),('xgb',xgb)] #list of (str, estimator)

lr = LogisticRegression()

stack_model = StackingClassifier( estimators = clf, final_estimator = lr)

print(stack_model.score(x_train,y_train))
print(stack_model.score(x_test,y_test))

AttributeError: This 'StackingClassifier' has no attribute 'predict'